In [1]:
import pickle
import numpy as np
import torch

In [2]:
import sys
sys.path.append('../')
from uniperceiver.tokenization import ClipTokenizer

## 1. ImageNet 1k class names

### Tokenize the class names

In [3]:
# We follow CLIP to use the ImageNet-1k class names sourced from Anish Athalye's imagenet-simple-labels.

class_names = ["tench", "goldfish", "great white shark", "tiger shark", "hammerhead shark", "electric ray", "stingray", "rooster", "hen", "ostrich", "brambling", "goldfinch", "house finch", "junco", "indigo bunting", "American robin", "bulbul", "jay", "magpie", "chickadee", "American dipper", "kite (bird of prey)", "bald eagle", "vulture", "great grey owl", "fire salamander", "smooth newt", "newt", "spotted salamander", "axolotl", "American bullfrog", "tree frog", "tailed frog", "loggerhead sea turtle", "leatherback sea turtle", "mud turtle", "terrapin", "box turtle", "banded gecko", "green iguana", "Carolina anole", "desert grassland whiptail lizard", "agama", "frilled-necked lizard", "alligator lizard", "Gila monster", "European green lizard", "chameleon", "Komodo dragon", "Nile crocodile", "American alligator", "triceratops", "worm snake", "ring-necked snake", "eastern hog-nosed snake", "smooth green snake", "kingsnake", "garter snake", "water snake", "vine snake", "night snake", "boa constrictor", "African rock python", "Indian cobra", "green mamba", "sea snake", "Saharan horned viper", "eastern diamondback rattlesnake", "sidewinder rattlesnake", "trilobite", "harvestman", "scorpion", "yellow garden spider", "barn spider", "European garden spider", "southern black widow", "tarantula", "wolf spider", "tick", "centipede", "black grouse", "ptarmigan", "ruffed grouse", "prairie grouse", "peafowl", "quail", "partridge", "african grey parrot", "macaw", "sulphur-crested cockatoo", "lorikeet", "coucal", "bee eater", "hornbill", "hummingbird", "jacamar", "toucan", "duck", "red-breasted merganser", "goose", "black swan", "tusker", "echidna", "platypus", "wallaby", "koala", "wombat", "jellyfish", "sea anemone", "brain coral", "flatworm", "nematode", "conch", "snail", "slug", "sea slug", "chiton", "chambered nautilus", "Dungeness crab", "rock crab", "fiddler crab", "red king crab", "American lobster", "spiny lobster", "crayfish", "hermit crab", "isopod", "white stork", "black stork", "spoonbill", "flamingo", "little blue heron", "great egret", "bittern bird", "crane bird", "limpkin", "common gallinule", "American coot", "bustard", "ruddy turnstone", "dunlin", "common redshank", "dowitcher", "oystercatcher", "pelican", "king penguin", "albatross", "grey whale", "killer whale", "dugong", "sea lion", "Chihuahua", "Japanese Chin", "Maltese", "Pekingese", "Shih Tzu", "King Charles Spaniel", "Papillon", "toy terrier", "Rhodesian Ridgeback", "Afghan Hound", "Basset Hound", "Beagle", "Bloodhound", "Bluetick Coonhound", "Black and Tan Coonhound", "Treeing Walker Coonhound", "English foxhound", "Redbone Coonhound", "borzoi", "Irish Wolfhound", "Italian Greyhound", "Whippet", "Ibizan Hound", "Norwegian Elkhound", "Otterhound", "Saluki", "Scottish Deerhound", "Weimaraner", "Staffordshire Bull Terrier", "American Staffordshire Terrier", "Bedlington Terrier", "Border Terrier", "Kerry Blue Terrier", "Irish Terrier", "Norfolk Terrier", "Norwich Terrier", "Yorkshire Terrier", "Wire Fox Terrier", "Lakeland Terrier", "Sealyham Terrier", "Airedale Terrier", "Cairn Terrier", "Australian Terrier", "Dandie Dinmont Terrier", "Boston Terrier", "Miniature Schnauzer", "Giant Schnauzer", "Standard Schnauzer", "Scottish Terrier", "Tibetan Terrier", "Australian Silky Terrier", "Soft-coated Wheaten Terrier", "West Highland White Terrier", "Lhasa Apso", "Flat-Coated Retriever", "Curly-coated Retriever", "Golden Retriever", "Labrador Retriever", "Chesapeake Bay Retriever", "German Shorthaired Pointer", "Vizsla", "English Setter", "Irish Setter", "Gordon Setter", "Brittany dog", "Clumber Spaniel", "English Springer Spaniel", "Welsh Springer Spaniel", "Cocker Spaniel", "Sussex Spaniel", "Irish Water Spaniel", "Kuvasz", "Schipperke", "Groenendael dog", "Malinois", "Briard", "Australian Kelpie", "Komondor", "Old English Sheepdog", "Shetland Sheepdog", "collie", "Border Collie", "Bouvier des Flandres dog", "Rottweiler", "German Shepherd Dog", "Dobermann", "Miniature Pinscher", "Greater Swiss Mountain Dog", "Bernese Mountain Dog", "Appenzeller Sennenhund", "Entlebucher Sennenhund", "Boxer", "Bullmastiff", "Tibetan Mastiff", "French Bulldog", "Great Dane", "St. Bernard", "husky", "Alaskan Malamute", "Siberian Husky", "Dalmatian", "Affenpinscher", "Basenji", "pug", "Leonberger", "Newfoundland dog", "Great Pyrenees dog", "Samoyed", "Pomeranian", "Chow Chow", "Keeshond", "brussels griffon", "Pembroke Welsh Corgi", "Cardigan Welsh Corgi", "Toy Poodle", "Miniature Poodle", "Standard Poodle", "Mexican hairless dog (xoloitzcuintli)", "grey wolf", "Alaskan tundra wolf", "red wolf or maned wolf", "coyote", "dingo", "dhole", "African wild dog", "hyena", "red fox", "kit fox", "Arctic fox", "grey fox", "tabby cat", "tiger cat", "Persian cat", "Siamese cat", "Egyptian Mau", "cougar", "lynx", "leopard", "snow leopard", "jaguar", "lion", "tiger", "cheetah", "brown bear", "American black bear", "polar bear", "sloth bear", "mongoose", "meerkat", "tiger beetle", "ladybug", "ground beetle", "longhorn beetle", "leaf beetle", "dung beetle", "rhinoceros beetle", "weevil", "fly", "bee", "ant", "grasshopper", "cricket insect", "stick insect", "cockroach", "praying mantis", "cicada", "leafhopper", "lacewing", "dragonfly", "damselfly", "red admiral butterfly", "ringlet butterfly", "monarch butterfly", "small white butterfly", "sulphur butterfly", "gossamer-winged butterfly", "starfish", "sea urchin", "sea cucumber", "cottontail rabbit", "hare", "Angora rabbit", "hamster", "porcupine", "fox squirrel", "marmot", "beaver", "guinea pig", "common sorrel horse", "zebra", "pig", "wild boar", "warthog", "hippopotamus", "ox", "water buffalo", "bison", "ram (adult male sheep)", "bighorn sheep", "Alpine ibex", "hartebeest", "impala (antelope)", "gazelle", "arabian camel", "llama", "weasel", "mink", "European polecat", "black-footed ferret", "otter", "skunk", "badger", "armadillo", "three-toed sloth", "orangutan", "gorilla", "chimpanzee", "gibbon", "siamang", "guenon", "patas monkey", "baboon", "macaque", "langur", "black-and-white colobus", "proboscis monkey", "marmoset", "white-headed capuchin", "howler monkey", "titi monkey", "Geoffroy's spider monkey", "common squirrel monkey", "ring-tailed lemur", "indri", "Asian elephant", "African bush elephant", "red panda", "giant panda", "snoek fish", "eel", "silver salmon", "rock beauty fish", "clownfish", "sturgeon", "gar fish", "lionfish", "pufferfish", "abacus", "abaya", "academic gown", "accordion", "acoustic guitar", "aircraft carrier", "airliner", "airship", "altar", "ambulance", "amphibious vehicle", "analog clock", "apiary", "apron", "trash can", "assault rifle", "backpack", "bakery", "balance beam", "balloon", "ballpoint pen", "Band-Aid", "banjo", "baluster / handrail", "barbell", "barber chair", "barbershop", "barn", "barometer", "barrel", "wheelbarrow", "baseball", "basketball", "bassinet", "bassoon", "swimming cap", "bath towel", "bathtub", "station wagon", "lighthouse", "beaker", "military hat (bearskin or shako)", "beer bottle", "beer glass", "bell tower", "baby bib", "tandem bicycle", "bikini", "ring binder", "binoculars", "birdhouse", "boathouse", "bobsleigh", "bolo tie", "poke bonnet", "bookcase", "bookstore", "bottle cap", "hunting bow", "bow tie", "brass memorial plaque", "bra", "breakwater", "breastplate", "broom", "bucket", "buckle", "bulletproof vest", "high-speed train", "butcher shop", "taxicab", "cauldron", "candle", "cannon", "canoe", "can opener", "cardigan", "car mirror", "carousel", "tool kit", "cardboard box / carton", "car wheel", "automated teller machine", "cassette", "cassette player", "castle", "catamaran", "CD player", "cello", "mobile phone", "chain", "chain-link fence", "chain mail", "chainsaw", "storage chest", "chiffonier", "bell or wind chime", "china cabinet", "Christmas stocking", "church", "movie theater", "cleaver", "cliff dwelling", "cloak", "clogs", "cocktail shaker", "coffee mug", "coffeemaker", "spiral or coil", "combination lock", "computer keyboard", "candy store", "container ship", "convertible", "corkscrew", "cornet", "cowboy boot", "cowboy hat", "cradle", "construction crane", "crash helmet", "crate", "infant bed", "Crock Pot", "croquet ball", "crutch", "cuirass", "dam", "desk", "desktop computer", "rotary dial telephone", "diaper", "digital clock", "digital watch", "dining table", "dishcloth", "dishwasher", "disc brake", "dock", "dog sled", "dome", "doormat", "drilling rig", "drum", "drumstick", "dumbbell", "Dutch oven", "electric fan", "electric guitar", "electric locomotive", "entertainment center", "envelope", "espresso machine", "face powder", "feather boa", "filing cabinet", "fireboat", "fire truck", "fire screen", "flagpole", "flute", "folding chair", "football helmet", "forklift", "fountain", "fountain pen", "four-poster bed", "freight car", "French horn", "frying pan", "fur coat", "garbage truck", "gas mask or respirator", "gas pump", "goblet", "go-kart", "golf ball", "golf cart", "gondola", "gong", "gown", "grand piano", "greenhouse", "radiator grille", "grocery store", "guillotine", "hair clip", "hair spray", "half-track", "hammer", "hamper", "hair dryer", "hand-held computer", "handkerchief", "hard disk drive", "harmonica", "harp", "combine harvester", "hatchet", "holster", "home theater", "honeycomb", "hook", "hoop skirt", "gymnastic horizontal bar", "horse-drawn vehicle", "hourglass", "iPod", "clothes iron", "carved pumpkin", "jeans", "jeep", "T-shirt", "jigsaw puzzle", "rickshaw", "joystick", "kimono", "knee pad", "knot", "lab coat", "ladle", "lampshade", "laptop computer", "lawn mower", "lens cap", "letter opener", "library", "lifeboat", "lighter", "limousine", "ocean liner", "lipstick", "slip-on shoe", "lotion", "music speaker", "loupe magnifying glass", "sawmill", "magnetic compass", "messenger bag", "mailbox", "tights", "one-piece bathing suit", "manhole cover", "maraca", "marimba", "mask", "matchstick", "maypole", "maze", "measuring cup", "medicine cabinet", "megalith", "microphone", "microwave oven", "military uniform", "milk can", "minibus", "miniskirt", "minivan", "missile", "mitten", "mixing bowl", "mobile home", "ford model t", "modem", "monastery", "monitor", "moped", "mortar and pestle", "graduation cap", "mosque", "mosquito net", "vespa", "mountain bike", "tent", "computer mouse", "mousetrap", "moving van", "muzzle", "metal nail", "neck brace", "necklace", "baby pacifier", "notebook computer", "obelisk", "oboe", "ocarina", "odometer", "oil filter", "pipe organ", "oscilloscope", "overskirt", "bullock cart", "oxygen mask", "product packet / packaging", "paddle", "paddle wheel", "padlock", "paintbrush", "pajamas", "palace", "pan flute", "paper towel", "parachute", "parallel bars", "park bench", "parking meter", "railroad car", "patio", "payphone", "pedestal", "pencil case", "pencil sharpener", "perfume", "Petri dish", "photocopier", "plectrum", "Pickelhaube", "picket fence", "pickup truck", "pier", "piggy bank", "pill bottle", "pillow", "ping-pong ball", "pinwheel", "pirate ship", "drink pitcher", "block plane", "planetarium", "plastic bag", "plate rack", "farm plow", "plunger", "Polaroid camera", "pole", "police van", "poncho", "pool table", "soda bottle", "plant pot", "potter's wheel", "power drill", "prayer rug", "printer", "prison", "missile", "projector", "hockey puck", "punching bag", "purse", "quill", "quilt", "race car", "racket", "radiator", "radio", "radio telescope", "rain barrel", "recreational vehicle", "fishing casting reel", "reflex camera", "refrigerator", "remote control", "restaurant", "revolver", "rifle", "rocking chair", "rotisserie", "eraser", "rugby ball", "ruler measuring stick", "sneaker", "safe", "safety pin", "salt shaker", "sandal", "sarong", "saxophone", "scabbard", "weighing scale", "school bus", "schooner", "scoreboard", "CRT monitor", "screw", "screwdriver", "seat belt", "sewing machine", "shield", "shoe store", "shoji screen / room divider", "shopping basket", "shopping cart", "shovel", "shower cap", "shower curtain", "ski", "balaclava ski mask", "sleeping bag", "slide rule", "sliding door", "slot machine", "snorkel", "snowmobile", "snowplow", "soap dispenser", "soccer ball", "sock", "solar thermal collector", "sombrero", "soup bowl", "keyboard space bar", "space heater", "space shuttle", "spatula", "motorboat", "spider web", "spindle", "sports car", "spotlight", "stage", "steam locomotive", "through arch bridge", "steel drum", "stethoscope", "scarf", "stone wall", "stopwatch", "stove", "strainer", "tram", "stretcher", "couch", "stupa", "submarine", "suit", "sundial", "sunglasses", "sunglasses", "sunscreen", "suspension bridge", "mop", "sweatshirt", "swim trunks / shorts", "swing", "electrical switch", "syringe", "table lamp", "tank", "tape player", "teapot", "teddy bear", "television", "tennis ball", "thatched roof", "front curtain", "thimble", "threshing machine", "throne", "tile roof", "toaster", "tobacco shop", "toilet seat", "torch", "totem pole", "tow truck", "toy store", "tractor", "semi-trailer truck", "tray", "trench coat", "tricycle", "trimaran", "tripod", "triumphal arch", "trolleybus", "trombone", "hot tub", "turnstile", "typewriter keyboard", "umbrella", "unicycle", "upright piano", "vacuum cleaner", "vase", "vaulted or arched ceiling", "velvet fabric", "vending machine", "vestment", "viaduct", "violin", "volleyball", "waffle iron", "wall clock", "wallet", "wardrobe", "military aircraft", "sink", "washing machine", "water bottle", "water jug", "water tower", "whiskey jug", "whistle", "hair wig", "window screen", "window shade", "Windsor tie", "wine bottle", "airplane wing", "wok", "wooden spoon", "wool", "split-rail fence", "shipwreck", "sailboat", "yurt", "website", "comic book", "crossword", "traffic or street sign", "traffic light", "dust jacket", "menu", "plate", "guacamole", "consomme", "hot pot", "trifle", "ice cream", "popsicle", "baguette", "bagel", "pretzel", "cheeseburger", "hot dog", "mashed potatoes", "cabbage", "broccoli", "cauliflower", "zucchini", "spaghetti squash", "acorn squash", "butternut squash", "cucumber", "artichoke", "bell pepper", "cardoon", "mushroom", "Granny Smith apple", "strawberry", "orange", "lemon", "fig", "pineapple", "banana", "jackfruit", "cherimoya (custard apple)", "pomegranate", "hay", "carbonara", "chocolate syrup", "dough", "meatloaf", "pizza", "pot pie", "burrito", "red wine", "espresso", "tea cup", "eggnog", "mountain", "bubble", "cliff", "coral reef", "geyser", "lakeshore", "promontory", "sandbar", "beach", "valley", "volcano", "baseball player", "bridegroom", "scuba diver", "rapeseed", "daisy", "yellow lady's slipper", "corn", "acorn", "rose hip", "horse chestnut seed", "coral fungus", "agaric", "gyromitra", "stinkhorn mushroom", "earth star fungus", "hen of the woods mushroom", "bolete", "corn cob", "toilet paper"]


In [4]:
tokenizer = ClipTokenizer()
class_names_tokenized = [torch.tensor(tokenizer.encode(x + " <|endoftext|>")) for x in class_names]

### Add the metadata

In [5]:
shared_target_set = {
    'data': class_names_tokenized,
    'modality': 'text'
}

### Save the target set

In [8]:
# This file location should be specified in the "SHARED_TARGETS" section of the config file.
with open('/nfs/zhujinguo/datasets/open_source_dataset/imagenet_class_name_CLIP_with_endoftext.pkl', 'wb') as f:
    pickle.dump(shared_target_set, f)

## 2. ImageNet 22k class names

In [4]:
# For ImageNet 22k, we use the ImageNet 1k class names as the first 1000 categories, and use the WordNet synonyms for the remaining categories.
with open('/nfs/zhujinguo/datasets/open_source_dataset/imagenet_22k_class_names.pkl', 'rb') as f:
    class_name_mapping = pickle.load(f)

class_names = [c[1] for c in class_name_mapping]

In [10]:
tokenizer = ClipTokenizer()
class_names_tokenized = [[torch.tensor(tokenizer.encode(x + " <|endoftext|>")) for x in c] for c in class_names]

### Add the metadata

In [12]:
shared_target_set = {
    'data': class_names_tokenized,
    'modality': 'text'
}

### Save the target set

In [13]:
# This file location should be specified in the "SHARED_TARGETS" section of the config file.
with open('/nfs/zhujinguo/datasets/open_source_dataset/imagenet_22k_class_name_CLIP_with_endoftext.pkl', 'wb') as f:
    pickle.dump(shared_target_set, f)

## 3. Vocabulary

### Tokenize the vocabulary

In [14]:
tokenizer = ClipTokenizer()
eot = tokenizer.encoder['<|endoftext|>']
vocabulary_tokenized = [torch.tensor([v, eot]) for v in tokenizer.encoder.values]

### Add the metadata

In [16]:
shared_target_set = {
    'data': vocabulary_tokenized,
    'modality': 'text'
}

### Save the target set

In [17]:
with open('/nfs/zhujinguo/datasets/open_source_dataset/vocabulary_CLIP_with_endoftext.pkl', 'wb') as f:
    pickle.dump(shared_target_set, f)

## 4. Kinetics-400 class names

### Tokenize the class names

In [ ]:
# Class names obtained from https://gist.github.com/willprice/f19da185c9c5f32847134b87c1960769

class_names = ['abseiling', 'air drumming', 'answering questions', 'applauding', 'applying cream', 'archery', 'arm wrestling', 'arranging flowers', 'assembling computer', 'auctioning', 'baby waking up', 'baking cookies', 'balloon blowing', 'bandaging', 'barbequing', 'bartending', 'beatboxing', 'bee keeping', 'belly dancing', 'bench pressing', 'bending back', 'bending metal', 'biking through snow', 'blasting sand', 'blowing glass', 'blowing leaves', 'blowing nose', 'blowing out candles', 'bobsledding', 'bookbinding', 'bouncing on trampoline', 'bowling', 'braiding hair', 'breading or breadcrumbing', 'breakdancing', 'brush painting', 'brushing hair', 'brushing teeth', 'building cabinet', 'building shed', 'bungee jumping', 'busking', 'canoeing or kayaking', 'capoeira', 'carrying baby', 'cartwheeling', 'carving pumpkin', 'catching fish', 'catching or throwing baseball', 'catching or throwing frisbee', 'catching or throwing softball', 'celebrating', 'changing oil', 'changing wheel', 'checking tires', 'cheerleading', 'chopping wood', 'clapping', 'clay pottery making', 'clean and jerk', 'cleaning floor', 'cleaning gutters', 'cleaning pool', 'cleaning shoes', 'cleaning toilet', 'cleaning windows', 'climbing a rope', 'climbing ladder', 'climbing tree', 'contact juggling', 'cooking chicken', 'cooking egg', 'cooking on campfire', 'cooking sausages', 'counting money', 'country line dancing', 'cracking neck', 'crawling baby', 'crossing river', 'crying', 'curling hair', 'cutting nails', 'cutting pineapple', 'cutting watermelon', 'dancing ballet', 'dancing charleston', 'dancing gangnam style', 'dancing macarena', 'deadlifting', 'decorating the christmas tree', 'digging', 'dining', 'disc golfing', 'diving cliff', 'dodgeball', 'doing aerobics', 'doing laundry', 'doing nails', 'drawing', 'dribbling basketball', 'drinking', 'drinking beer', 'drinking shots', 'driving car', 'driving tractor', 'drop kicking', 'drumming fingers', 'dunking basketball', 'dying hair', 'eating burger', 'eating cake', 'eating carrots', 'eating chips', 'eating doughnuts', 'eating hotdog', 'eating ice cream', 'eating spaghetti', 'eating watermelon', 'egg hunting', 'exercising arm', 'exercising with an exercise ball', 'extinguishing fire', 'faceplanting', 'feeding birds', 'feeding fish', 'feeding goats', 'filling eyebrows', 'finger snapping', 'fixing hair', 'flipping pancake', 'flying kite', 'folding clothes', 'folding napkins', 'folding paper', 'front raises', 'frying vegetables', 'garbage collecting', 'gargling', 'getting a haircut', 'getting a tattoo', 'giving or receiving award', 'golf chipping', 'golf driving', 'golf putting', 'grinding meat', 'grooming dog', 'grooming horse', 'gymnastics tumbling', 'hammer throw', 'headbanging', 'headbutting', 'high jump', 'high kick', 'hitting baseball', 'hockey stop', 'holding snake', 'hopscotch', 'hoverboarding', 'hugging', 'hula hooping', 'hurdling', 'hurling (sport)', 'ice climbing', 'ice fishing', 'ice skating', 'ironing', 'javelin throw', 'jetskiing', 'jogging', 'juggling balls', 'juggling fire', 'juggling soccer ball', 'jumping into pool', 'jumpstyle dancing', 'kicking field goal', 'kicking soccer ball', 'kissing', 'kitesurfing', 'knitting', 'krumping', 'laughing', 'laying bricks', 'long jump', 'lunge', 'making a cake', 'making a sandwich', 'making bed', 'making jewelry', 'making pizza', 'making snowman', 'making sushi', 'making tea', 'marching', 'massaging back', 'massaging feet', 'massaging legs', "massaging person's head", 'milking cow', 'mopping floor', 'motorcycling', 'moving furniture', 'mowing lawn', 'news anchoring', 'opening bottle', 'opening present', 'paragliding', 'parasailing', 'parkour', 'passing American football (in game)', 'passing American football (not in game)', 'peeling apples', 'peeling potatoes', 'petting animal (not cat)', 'petting cat', 'picking fruit', 'planting trees', 'plastering', 'playing accordion', 'playing badminton', 'playing bagpipes', 'playing basketball', 'playing bass guitar', 'playing cards', 'playing cello', 'playing chess', 'playing clarinet', 'playing controller', 'playing cricket', 'playing cymbals', 'playing didgeridoo', 'playing drums', 'playing flute', 'playing guitar', 'playing harmonica', 'playing harp', 'playing ice hockey', 'playing keyboard', 'playing kickball', 'playing monopoly', 'playing organ', 'playing paintball', 'playing piano', 'playing poker', 'playing recorder', 'playing saxophone', 'playing squash or racquetball', 'playing tennis', 'playing trombone', 'playing trumpet', 'playing ukulele', 'playing violin', 'playing volleyball', 'playing xylophone', 'pole vault', 'presenting weather forecast', 'pull ups', 'pumping fist', 'pumping gas', 'punching bag', 'punching person (boxing)', 'push up', 'pushing car', 'pushing cart', 'pushing wheelchair', 'reading book', 'reading newspaper', 'recording music', 'riding a bike', 'riding camel', 'riding elephant', 'riding mechanical bull', 'riding mountain bike', 'riding mule', 'riding or walking with horse', 'riding scooter', 'riding unicycle', 'ripping paper', 'robot dancing', 'rock climbing', 'rock scissors paper', 'roller skating', 'running on treadmill', 'sailing', 'salsa dancing', 'sanding floor', 'scrambling eggs', 'scuba diving', 'setting table', 'shaking hands', 'shaking head', 'sharpening knives', 'sharpening pencil', 'shaving head', 'shaving legs', 'shearing sheep', 'shining shoes', 'shooting basketball', 'shooting goal (soccer)', 'shot put', 'shoveling snow', 'shredding paper', 'shuffling cards', 'side kick', 'sign language interpreting', 'singing', 'situp', 'skateboarding', 'ski jumping', 'skiing (not slalom or crosscountry)', 'skiing crosscountry', 'skiing slalom', 'skipping rope', 'skydiving', 'slacklining', 'slapping', 'sled dog racing', 'smoking', 'smoking hookah', 'snatch weight lifting', 'sneezing', 'sniffing', 'snorkeling', 'snowboarding', 'snowkiting', 'snowmobiling', 'somersaulting', 'spinning poi', 'spray painting', 'spraying', 'springboard diving', 'squat', 'sticking tongue out', 'stomping grapes', 'stretching arm', 'stretching leg', 'strumming guitar', 'surfing crowd', 'surfing water', 'sweeping floor', 'swimming backstroke', 'swimming breast stroke', 'swimming butterfly stroke', 'swing dancing', 'swinging legs', 'swinging on something', 'sword fighting', 'tai chi', 'taking a shower', 'tango dancing', 'tap dancing', 'tapping guitar', 'tapping pen', 'tasting beer', 'tasting food', 'testifying', 'texting', 'throwing axe', 'throwing ball', 'throwing discus', 'tickling', 'tobogganing', 'tossing coin', 'tossing salad', 'training dog', 'trapezing', 'trimming or shaving beard', 'trimming trees', 'triple jump', 'tying bow tie', 'tying knot (not on a tie)', 'tying tie', 'unboxing', 'unloading truck', 'using computer', 'using remote controller (not gaming)', 'using segway', 'vault', 'waiting in line', 'walking the dog', 'washing dishes', 'washing feet', 'washing hair', 'washing hands', 'water skiing', 'water sliding', 'watering plants', 'waxing back', 'waxing chest', 'waxing eyebrows', 'waxing legs', 'weaving basket', 'welding', 'whistling', 'windsurfing', 'wrapping present', 'wrestling', 'writing', 'yawning', 'yoga', 'zumba']

In [ ]:
tokenizer = ClipTokenizer()
class_names_tokenized = [torch.tensor(tokenizer.encode(x + " <|endoftext|>")) for x in class_names]

### Add the metadata

In [ ]:
shared_target_set = {
    'data': class_names_tokenized,
    'modality': 'text'
}

### Save the target set

In [ ]:
# This file location should be specified in the "SHARED_TARGETS" section of the config file.
with open('/nfs/zhujinguo/datasets/open_source_dataset/k400_class_name_CLIP_with_endoftext.pkl', 'wb') as f:
    pickle.dump(shared_target_set, f)

## 5. Kinetics-700 class names

### Tokenize the class names

In [8]:
# Class names obtained from https://gist.github.com/willprice/f19da185c9c5f32847134b87c1960769

class_names = ['abseiling', 'acting in play', 'adjusting glasses', 'air drumming', 'alligator wrestling', 'answering questions', 'applauding', 'applying cream', 'archaeological excavation', 'archery', 'arguing', 'arm wrestling', 'arranging flowers', 'arresting', 'assembling bicycle', 'assembling computer', 'attending conference', 'auctioning', 'baby waking up', 'backflip (human)', 'baking cookies', 'bandaging', 'barbequing', 'bartending', 'base jumping', 'bathing dog', 'battle rope training', 'beatboxing', 'bee keeping', 'being excited', 'being in zero gravity', 'belly dancing', 'bench pressing', 'bending back', 'bending metal', 'biking through snow', 'blasting sand', 'blending fruit', 'blowdrying hair', 'blowing bubble gum', 'blowing glass', 'blowing leaves', 'blowing nose', 'blowing out candles', 'bobsledding', 'bodysurfing', 'bookbinding', 'bottling', 'bouncing ball (not juggling)', 'bouncing on bouncy castle', 'bouncing on trampoline', 'bowling', 'braiding hair', 'breading or breadcrumbing', 'breakdancing', 'breaking boards', 'breaking glass', 'breathing fire', 'brush painting', 'brushing floor', 'brushing hair', 'brushing teeth', 'building cabinet', 'building lego', 'building sandcastle', 'building shed', 'bulldozing', 'bungee jumping', 'burping', 'busking', 'calculating', 'calligraphy', 'canoeing or kayaking', 'capoeira', 'capsizing', 'card stacking', 'card throwing', 'carrying baby', 'carrying weight', 'cartwheeling', 'carving ice', 'carving marble', 'carving pumpkin', 'carving wood with a knife', 'casting fishing line', 'catching fish', 'catching or throwing baseball', 'catching or throwing frisbee', 'catching or throwing softball', 'celebrating', 'changing gear in car', 'changing oil', 'changing wheel (not on bike)', 'chasing', 'checking tires', 'checking watch', 'cheerleading', 'chewing gum', 'chiseling stone', 'chiseling wood', 'chopping meat', 'chopping wood', 'clam digging', 'clapping', 'clay pottery making', 'clean and jerk', 'cleaning gutters', 'cleaning pool', 'cleaning shoes', 'cleaning toilet', 'cleaning windows', 'climbing a rope', 'climbing ladder', 'climbing tree', 'closing door', 'coloring in', 'combing hair', 'contact juggling', 'contorting', 'cooking chicken', 'cooking egg', 'cooking on campfire', 'cooking sausages (not on barbeque)', 'cooking scallops', 'cosplaying', 'coughing', 'counting money', 'country line dancing', 'cracking back', 'cracking knuckles', 'cracking neck', 'crawling baby', 'crocheting', 'crossing eyes', 'crossing river', 'crying', 'cumbia', 'curling (sport)', 'curling eyelashes', 'curling hair', 'cutting apple', 'cutting cake', 'cutting nails', 'cutting orange', 'cutting pineapple', 'cutting watermelon', 'dancing ballet', 'dancing charleston', 'dancing gangnam style', 'dancing macarena', 'deadlifting', 'dealing cards', 'decorating the christmas tree', 'decoupage', 'delivering mail', 'digging', 'dining', 'directing traffic', 'disc golfing', 'diving cliff', 'docking boat', 'dodgeball', 'doing aerobics', 'doing jigsaw puzzle', 'doing laundry', 'doing nails', 'doing sudoku', 'drawing', 'dribbling basketball', 'drinking shots', 'driving car', 'driving tractor', 'drooling', 'drop kicking', 'drumming fingers', 'dumpster diving', 'dunking basketball', 'dyeing eyebrows', 'dyeing hair', 'eating burger', 'eating cake', 'eating carrots', 'eating chips', 'eating doughnuts', 'eating hotdog', 'eating ice cream', 'eating nachos', 'eating spaghetti', 'eating watermelon', 'egg hunting', 'embroidering', 'entering church', 'exercising arm', 'exercising with an exercise ball', 'extinguishing fire', 'faceplanting', 'falling off bike', 'falling off chair', 'feeding birds', 'feeding fish', 'feeding goats', 'fencing (sport)', 'fidgeting', 'filling cake', 'filling eyebrows', 'finger snapping', 'fixing bicycle', 'fixing hair', 'flint knapping', 'flipping bottle', 'flipping pancake', 'fly tying', 'flying kite', 'folding clothes', 'folding napkins', 'folding paper', 'front raises', 'frying vegetables', 'gargling', 'geocaching', 'getting a haircut', 'getting a piercing', 'getting a tattoo', 'giving or receiving award', 'gold panning', 'golf chipping', 'golf driving', 'golf putting', 'gospel singing in church', 'grinding meat', 'grooming cat', 'grooming dog', 'grooming horse', 'gymnastics tumbling', 'hammer throw', 'hand washing clothes', 'head stand', 'headbanging', 'headbutting', 'helmet diving', 'herding cattle', 'high fiving', 'high jump', 'high kick', 'historical reenactment', 'hitting baseball', 'hockey stop', 'holding snake', 'home roasting coffee', 'hopscotch', 'hoverboarding', 'huddling', 'hugging (not baby)', 'hugging baby', 'hula hooping', 'hurdling', 'hurling (sport)', 'ice climbing', 'ice fishing', 'ice skating', 'ice swimming', 'inflating balloons', 'installing carpet', 'ironing', 'ironing hair', 'javelin throw', 'jaywalking', 'jetskiing', 'jogging', 'juggling balls', 'juggling fire', 'juggling soccer ball', 'jumping bicycle', 'jumping into pool', 'jumping jacks', 'jumping sofa', 'jumpstyle dancing', 'karaoke', 'kicking field goal', 'kicking soccer ball', 'kissing', 'kitesurfing', 'knitting', 'krumping', 'land sailing', 'laughing', 'lawn mower racing', 'laying bricks', 'laying concrete', 'laying decking', 'laying stone', 'laying tiles', 'leatherworking', 'letting go of balloon', 'licking', 'lifting hat', 'lighting candle', 'lighting fire', 'listening with headphones', 'lock picking', 'long jump', 'longboarding', 'looking at phone', 'looking in mirror', 'luge', 'lunge', 'making a cake', 'making a sandwich', 'making balloon shapes', 'making bubbles', 'making cheese', 'making horseshoes', 'making jewelry', 'making latte art', 'making paper aeroplanes', 'making pizza', 'making slime', 'making snowman', 'making sushi', 'making tea', 'making the bed', 'marching', 'marriage proposal', 'massaging back', 'massaging feet', 'massaging legs', 'massaging neck', "massaging person's head", 'metal detecting', 'milking cow', 'milking goat', 'mixing colours', 'moon walking', 'mopping floor', 'mosh pit dancing', 'motorcycling', 'mountain climber (exercise)', 'moving baby', 'moving child', 'moving furniture', 'mowing lawn', 'mushroom foraging', 'needle felting', 'news anchoring', 'opening bottle (not wine)', 'opening coconuts', 'opening door', 'opening present', 'opening refrigerator', 'opening wine bottle', 'packing', 'paragliding', 'parasailing', 'parkour', 'passing American football (in game)', 'passing American football (not in game)', 'passing soccer ball', 'peeling apples', 'peeling banana', 'peeling potatoes', 'person collecting garbage', 'petting animal (not cat)', 'petting cat', 'petting horse', 'photobombing', 'photocopying', 'picking apples', 'picking blueberries', 'pillow fight', 'pinching', 'pirouetting', 'planing wood', 'planting trees', 'plastering', 'playing accordion', 'playing american football', 'playing badminton', 'playing bagpipes', 'playing basketball', 'playing bass guitar', 'playing beer pong', 'playing billiards', 'playing blackjack', 'playing cards', 'playing cello', 'playing checkers', 'playing chess', 'playing clarinet', 'playing controller', 'playing cricket', 'playing cymbals', 'playing darts', 'playing didgeridoo', 'playing dominoes', 'playing drums', 'playing field hockey', 'playing flute', 'playing gong', 'playing guitar', 'playing hand clapping games', 'playing harmonica', 'playing harp', 'playing ice hockey', 'playing keyboard', 'playing kickball', 'playing laser tag', 'playing lute', 'playing mahjong', 'playing maracas', 'playing marbles', 'playing monopoly', 'playing netball', 'playing nose flute', 'playing oboe', 'playing ocarina', 'playing organ', 'playing paintball', 'playing pan pipes', 'playing piano', 'playing piccolo', 'playing pinball', 'playing ping pong', 'playing poker', 'playing polo', 'playing recorder', 'playing road hockey', 'playing rounders', 'playing rubiks cube', 'playing saxophone', 'playing scrabble', 'playing shuffleboard', 'playing slot machine', 'playing squash or racquetball', 'playing tennis', 'playing trombone', 'playing trumpet', 'playing ukulele', 'playing violin', 'playing volleyball', 'playing with trains', 'playing xylophone', 'poaching eggs', 'poking bellybutton', 'pole vault', 'polishing furniture', 'polishing metal', 'popping balloons', 'pouring beer', 'pouring milk', 'pouring wine', 'preparing salad', 'presenting weather forecast', 'pretending to be a statue', 'pull ups', 'pulling espresso shot', 'pulling rope (game)', 'pumping fist', 'pumping gas', 'punching bag', 'punching person (boxing)', 'push up', 'pushing car', 'pushing cart', 'pushing wheelbarrow', 'pushing wheelchair', 'putting in contact lenses', 'putting on eyeliner', 'putting on foundation', 'putting on lipstick', 'putting on mascara', 'putting on sari', 'putting on shoes', 'putting wallpaper on wall', 'raising eyebrows', 'reading book', 'reading newspaper', 'recording music', 'repairing puncture', 'riding a bike', 'riding camel', 'riding elephant', 'riding mechanical bull', 'riding mule', 'riding or walking with horse', 'riding scooter', 'riding snow blower', 'riding unicycle', 'ripping paper', 'roasting marshmallows', 'roasting pig', 'robot dancing', 'rock climbing', 'rock scissors paper', 'roller skating', 'rolling eyes', 'rolling pastry', 'rope pushdown', 'running on treadmill', 'sailing', 'salsa dancing', 'saluting', 'sanding floor', 'sanding wood', 'sausage making', 'sawing wood', 'scrambling eggs', 'scrapbooking', 'scrubbing face', 'scuba diving', 'seasoning food', 'separating eggs', 'setting table', 'sewing', 'shaking hands', 'shaking head', 'shaping bread dough', 'sharpening knives', 'sharpening pencil', 'shaving head', 'shaving legs', 'shearing sheep', 'shining flashlight', 'shining shoes', 'shoot dance', 'shooting basketball', 'shooting goal (soccer)', 'shooting off fireworks', 'shopping', 'shot put', 'shouting', 'shoveling snow', 'shredding paper', 'shucking oysters', 'shuffling cards', 'shuffling feet', 'side kick', 'sieving', 'sign language interpreting', 'silent disco', 'singing', 'sipping cup', 'situp', 'skateboarding', 'ski ballet', 'ski jumping', 'skiing crosscountry', 'skiing mono', 'skiing slalom', 'skipping rope', 'skipping stone', 'skydiving', 'slacklining', 'slapping', 'sled dog racing', 'sleeping', 'slicing onion', 'smashing', 'smelling feet', 'smoking', 'smoking hookah', 'smoking pipe', 'snatch weight lifting', 'sneezing', 'snorkeling', 'snowboarding', 'snowkiting', 'snowmobiling', 'somersaulting', 'spelunking', 'spinning plates', 'spinning poi', 'splashing water', 'spray painting', 'spraying', 'springboard diving', 'square dancing', 'squat', 'squeezing orange', 'stacking cups', 'stacking dice', 'standing on hands', 'staring', 'steer roping', 'steering car', 'sticking tongue out', 'stomping grapes', 'stretching arm', 'stretching leg', 'sucking lolly', 'surfing crowd', 'surfing water', 'surveying', 'sweeping floor', 'swimming backstroke', 'swimming breast stroke', 'swimming butterfly stroke', 'swimming front crawl', 'swimming with dolphins', 'swimming with sharks', 'swing dancing', 'swinging baseball bat', 'swinging on something', 'sword fighting', 'sword swallowing', 'tackling', 'tagging graffiti', 'tai chi', 'taking photo', 'talking on cell phone', 'tango dancing', 'tap dancing', 'tapping guitar', 'tapping pen', 'tasting beer', 'tasting food', 'tasting wine', 'testifying', 'texting', 'threading needle', 'throwing axe', 'throwing ball (not baseball or American football)', 'throwing discus', 'throwing knife', 'throwing snowballs', 'throwing tantrum', 'throwing water balloon', 'tickling', 'tie dying', 'tightrope walking', 'tiptoeing', 'tobogganing', 'tossing coin', 'tossing salad', 'training dog', 'trapezing', 'treating wood', 'trimming or shaving beard', 'trimming shrubs', 'trimming trees', 'triple jump', 'twiddling fingers', 'tying bow tie', 'tying knot (not on a tie)', 'tying necktie', 'tying shoe laces', 'unboxing', 'uncorking champagne', 'unloading truck', 'using a microscope', 'using a paint roller', 'using a power drill', 'using a sledge hammer', 'using a wrench', 'using atm', 'using bagging machine', 'using circular saw', 'using inhaler', 'using megaphone', 'using puppets', 'using remote controller (not gaming)', 'using segway', 'vacuuming car', 'vacuuming floor', 'visiting the zoo', 'wading through mud', 'wading through water', 'waiting in line', 'waking up', 'walking on stilts', 'walking the dog', 'walking through snow', 'walking with crutches', 'washing dishes', 'washing feet', 'washing hair', 'washing hands', 'watching tv', 'water skiing', 'water sliding', 'watering plants', 'waving hand', 'waxing armpits', 'waxing back', 'waxing chest', 'waxing eyebrows', 'waxing legs', 'weaving basket', 'weaving fabric', 'welding', 'whistling', 'windsurfing', 'winking', 'wood burning (art)', 'wrapping present', 'wrestling', 'writing', 'yarn spinning', 'yawning', 'yoga', 'zumba']

In [9]:
tokenizer = ClipTokenizer()
class_names_tokenized = [torch.tensor(tokenizer.encode(x + " <|endoftext|>")) for x in class_names]

### Add the metadata

In [10]:
shared_target_set = {
    'data': class_names_tokenized,
    'modality': 'text'
}

### Save the target set

In [11]:
# This file location should be specified in the "SHARED_TARGETS" section of the config file.
with open('/nfs/zhujinguo/datasets/open_source_dataset/k700_class_name_CLIP_with_endoftext.pkl', 'wb') as f:
    pickle.dump(shared_target_set, f)

## 6. Moments in time class names

In [21]:
class_names = ['adult female singing', 'adult female speaking', 'adult male singing', 'adult male speaking', 'aiming', 'applauding', 'arresting', 'ascending', 'asking', 'assembling', 'attacking', 'autographing', 'baking', 'balancing', 'baptizing', 'barbecuing', 'barking', 'bathing', 'bending', 'bicycling', 'biting', 'blocking', 'blowing', 'boarding', 'boating', 'boiling', 'bouncing', 'bowing', 'bowling', 'boxing', 'breaking', 'brushing', 'bubbling', 'building', 'bulldozing', 'burning', 'burying', 'buttoning', 'buying', 'calling', 'camping', 'carrying', 'carving', 'catching', 'celebrating', 'chasing', 'cheering', 'cheerleading', 'chewing', 'child singing', 'child speaking', 'chopping', 'clapping', 'clawing', 'cleaning', 'clearing', 'climbing', 'clinging', 'clipping', 'closing', 'coaching', 'colliding', 'combing', 'combusting', 'competing', 'constructing', 'cooking', 'coughing', 'covering', 'cracking', 'crafting', 'cramming', 'crashing', 'crawling', 'crouching', 'crushing', 'crying', 'cuddling', 'cutting', 'dancing', 'descending', 'destroying', 'digging', 'dining', 'dipping', 'discussing', 'diving', 'dragging', 'draining', 'drawing', 'drenching', 'dressing', 'drilling', 'drinking', 'dripping', 'driving', 'dropping', 'drumming', 'drying', 'dunking', 'dusting', 'eating', 'emptying', 'entering', 'erupting', 'exercising', 'exiting', 'extinguishing', 'falling', 'feeding', 'fencing', 'fighting', 'filling', 'filming', 'fishing', 'flicking', 'flipping', 'floating', 'flooding', 'flowing', 'flying', 'folding', 'frowning', 'frying', 'fueling', 'gambling', 'gardening', 'giggling', 'giving', 'grilling', 'grinning', 'gripping', 'grooming', 'guarding', 'hammering', 'handcuffing', 'handwriting', 'hanging', 'hiking', 'hitchhiking', 'hitting', 'howling', 'hugging', 'hunting', 'imitating', 'inflating', 'injecting', 'instructing', 'interviewing', 'jogging', 'joining', 'juggling', 'jumping', 'kicking', 'kissing', 'kneeling', 'knitting', 'knocking', 'landing', 'laughing', 'launching', 'leaking', 'leaning', 'leaping', 'lecturing', 'licking', 'lifting', 'loading', 'locking', 'manicuring', 'marching', 'marrying', 'massaging', 'measuring', 'mopping', 'mowing', 'officiating', 'opening', 'operating', 'overflowing', 'packaging', 'packing', 'painting', 'parading', 'paying', 'pedaling', 'peeling', 'performing', 'photographing', 'picking', 'piloting', 'pitching', 'placing', 'planting', 'playing', 'playing fun', 'playing music', 'playing sports', 'playing videogames', 'plugging', 'plunging', 'pointing', 'poking', 'pouring', 'praying', 'preaching', 'pressing', 'protesting', 'pulling', 'punching', 'punting', 'pushing', 'putting', 'queuing', 'racing', 'rafting', 'raining', 'raising', 'reaching', 'reading', 'removing', 'repairing', 'resting', 'riding', 'rinsing', 'rising', 'roaring', 'rocking', 'rolling', 'rowing', 'rubbing', 'running', 'sailing', 'saluting', 'sanding', 'sawing', 'scratching', 'screwing', 'scrubbing', 'selling', 'serving', 'sewing', 'shaking', 'shaving', 'shooting', 'shopping', 'shouting', 'shoveling', 'shredding', 'shrugging', 'signing', 'singing', 'sitting', 'skating', 'sketching', 'skiing', 'skipping', 'slapping', 'sleeping', 'slicing', 'sliding', 'slipping', 'smashing', 'smelling', 'smiling', 'smoking', 'snapping', 'sneezing', 'sniffing', 'snowing', 'snuggling', 'socializing', 'sowing', 'speaking', 'spilling', 'spinning', 'spitting', 'splashing', 'spraying', 'spreading', 'sprinkling', 'sprinting', 'squatting', 'squinting', 'stacking', 'standing', 'starting', 'stealing', 'steering', 'stirring', 'stitching', 'stomping', 'stopping', 'storming', 'stretching', 'stroking', 'studying', 'submerging', 'surfing', 'sweeping', 'swerving', 'swimming', 'swinging', 'talking', 'taping', 'tapping', 'tattooing', 'teaching', 'tearing', 'telephoning', 'throwing', 'tickling', 'towing', 'trimming', 'tripping', 'tuning', 'turning', 'twisting', 'tying', 'typing', 'unloading', 'unpacking', 'vacuuming', 'waking', 'walking', 'washing', 'watering', 'waving', 'waxing', 'weeding', 'welding', 'wetting', 'whistling', 'winking', 'working', 'wrapping', 'wrestling', 'writing', 'yawning']

In [22]:
tokenizer = ClipTokenizer()
class_names_tokenized = [torch.tensor(tokenizer.encode(x + " <|endoftext|>")) for x in class_names]

### Add the metadata

In [23]:
shared_target_set = {
    'data': class_names_tokenized,
    'modality': 'text'
}

### Save the target set

In [ ]:
# This file location should be specified in the "SHARED_TARGETS" section of the config file.
with open('/nfs/zhujinguo/datasets/open_source_dataset/MiT_class_name_CLIP_with_endoftext.pkl', 'wb') as f:
    pickle.dump(shared_target_set, f)

## 7.GLUE datasets

In [ ]:
# the order is crucial.
class_names_dict =  {
    'SST-2': ["bad",  "wonderful"],
    "CoLA":  [ "incorrect", "correct" ],
    "RTE":   [ "no", "yes",],
    "MRPC":  [ "no", "yes",],
    "QQP":   [ "no", "yes",],
    "QNLI":  [ "no", "yes",],
    "MNLI":  ["no", "maybe", "yes"]
}
tokenizer = ClipTokenizer()

In [ ]:
for dataset_name, class_names in class_names_dict.items():
    print(f'{dataset_name}, {class_names}')
    class_names_tokenized = [torch.tensor(tokenizer.encode(x + " <|endoftext|>")) for x in class_names]
    # Add the metadata
    shared_target_set = {
    'data': class_names_tokenized,
    'modality': 'text'
    }
    # This file location should be specified in the "SHARED_TARGETS" section of the config file.
    with open(f'/nfs/zhujinguo/datasets/open_source_dataset/bert_pretrain_data/glue_data/{dataset_name}_class_name_CLIP_with_endoftext.pkl', 'wb') as f:
        pickle.dump(shared_target_set, f)

## 8.VQA All Answers 

In [9]:
with open('/nfs/zhujinguo/datasets/open_source_dataset/VQA/trainval_label2ans.pkl', 'rb') as f:
    answers_mapping = pickle.load(f)

['',
 'woods',
 'trash can',
 'hanging',
 'wooden',
 'cooking',
 'china',
 'kids',
 'bike rack',
 'on phone',
 'music',
 'travel',
 'tulip',
 'arrow',
 'branch',
 'chevron',
 'mouth',
 'on right',
 'rice',
 'plate',
 'lots',
 'nature',
 'fruits',
 'throwing frisbee',
 'blonde',
 'life jacket',
 'ham',
 'hay',
 'hat',
 'to get to other side',
 '12:35',
 'shadow',
 '12:30',
 'crown',
 'crows',
 'bottom',
 'fish',
 'benches',
 'fabric',
 'congratulations',
 'passenger',
 'ana',
 'triangles',
 'roll',
 'chain',
 'chair',
 'crates',
 '11:45',
 'shirts',
 'mirrors',
 'parachute',
 'surfing',
 'mango',
 'years',
 'apron',
 'entering',
 '6:20',
 '6:25',
 'in her hand',
 'night time',
 'zoo',
 'printer',
 'suitcases',
 's',
 '4:20',
 'wine tasting',
 'west',
 'newspaper',
 'wires',
 'singapore',
 'advertisement',
 'for photo',
 'multi colored',
 'steamed',
 'traffic',
 'brushing',
 'stagecoach',
 'thailand',
 'pelicans',
 'pull',
 'dirty',
 'rust',
 'tennis shoes',
 'cream',
 'puppy',
 'in box'

In [6]:
tokenizer = ClipTokenizer()
answer_tokenized = [torch.tensor(tokenizer.encode(x + " <|endoftext|>")) for x in answers_mapping]

### Add meta data

In [7]:
shared_target_set = {
    'data': answer_tokenized,
    'modality': 'text'
}

### Save the target set

In [8]:
# This file location should be specified in the "SHARED_TARGETS" section of the config file.
with open('/nfs/zhujinguo/datasets/open_source_dataset/VQA_Answers_CLIP_with_endoftext.pkl', 'wb') as f:
    pickle.dump(shared_target_set, f)